## Informações Gerais
 | Informações | Detalhes |
 |------------|-------------|
 |Nome Tabela | gold.venda_total_produto |
 |Origem | silver.pedidos |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |14/09/2024 | Daniel Coutinho  | Criação do notebook |

In [0]:
%run /Projeto_Ampev/00.config/Configurações

In [0]:
# Importações
from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
# Nome do esquema e tabela no catálogo
database = "gold"
tabela = "venda_total_produto"

In [0]:
# Funções

#Função que aplica os comentários na tabela
def adicionaComentariosTabela(a,b,c,d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS '{c}'")
    for key,value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
#criação do df_produto com dados necessários para responder as perguntas do negócio
df_produto = spark.sql(
f""" 
SELECT
    produto AS nome_produto,
    CAST(SUM(quantidade) AS INT) AS quantidade_total,
    CAST(SUM(preco) AS DECIMAL(20,2)) AS valor_total
FROM
    silver.pedidos
GROUP BY all
""")

In [0]:
# incluir colunas de controle
df_produto = df_produto.withColumn("data_carga", current_date())
df_produto = df_produto.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))

In [0]:
#Comentario Tabela

comentario_tabela = 'Entidade contendo os pedidos mais vendidos'

lista_comentario_colunas = {
'nome_produto' : 'Nome do produto.',
'quantidade_total' : 'Quantidade total vendida.',
'valor_total' : 'Valor total vendido',
'data_carga' : 'Data que o registro foi carregado',
'data_hora_carga' : 'Data e hora que o registro foi carregado'
}

In [0]:
# Salvar o DataFrame no formato Delta
df_produto.write \
          .format('delta') \
          .mode('overwrite') \
          .clusterBy("nome_produto") \
          .option('overwriteSchema', 'true') \
          .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database, tabela, comentario_tabela, lista_comentario_colunas)
print("Dados gravados com sucesso!")

Dados gravados com sucesso!


In [0]:
# Otimização das tabelas
spark.sql(f"OPTIMIZE {database}.{tabela}")
print(f"Processo de otimização finalizado!.")

Processo de otimização finalizado!.


In [0]:
%sql
SELECT *
FROM gold.venda_total_produto

nome_produto,quantidade_total,valor_total,data_carga,data_hora_carga
Suco Natural,12931,444.03,2024-09-19,2024-09-19T19:51:46.271Z
Vinho ABC,14191,614.80,2024-09-19,2024-09-19T19:51:46.271Z
Refrigerante 123,11741,459.77,2024-09-19,2024-09-19T19:51:46.271Z
Cerveja XYZ,15036,606.78,2024-09-19,2024-09-19T19:51:46.271Z
Água Mineral,12874,533.71,2024-09-19,2024-09-19T19:51:46.271Z


In [0]:
%sql
--Quais são os produtos mais vendidos?
SELECT *
FROM gold.venda_total_produto
ORDER BY quantidade_total DESC limit 5

nome_produto,quantidade_total,valor_total,data_carga,data_hora_carga
Cerveja XYZ,15036,606.78,2024-09-19,2024-09-19T19:51:46.271Z
Vinho ABC,14191,614.80,2024-09-19,2024-09-19T19:51:46.271Z
Suco Natural,12931,444.03,2024-09-19,2024-09-19T19:51:46.271Z
Água Mineral,12874,533.71,2024-09-19,2024-09-19T19:51:46.271Z
Refrigerante 123,11741,459.77,2024-09-19,2024-09-19T19:51:46.271Z


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
--Quais são os produtos que mais geraram faturamento?
SELECT *
FROM gold.venda_total_produto
ORDER BY valor_total DESC limit 5

nome_produto,quantidade_total,valor_total,data_carga,data_hora_carga
Vinho ABC,14191,614.80,2024-09-19,2024-09-19T19:51:46.271Z
Cerveja XYZ,15036,606.78,2024-09-19,2024-09-19T19:51:46.271Z
Água Mineral,12874,533.71,2024-09-19,2024-09-19T19:51:46.271Z
Refrigerante 123,11741,459.77,2024-09-19,2024-09-19T19:51:46.271Z
Suco Natural,12931,444.03,2024-09-19,2024-09-19T19:51:46.271Z


Databricks visualization. Run in Databricks to view.